<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Submit-Form-as-Get-with-Cookies" data-toc-modified-id="Submit-Form-as-Get-with-Cookies-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Submit Form as Get with Cookies</a></span><ul class="toc-item"><li><span><a href="#Find-our-Cookie" data-toc-modified-id="Find-our-Cookie-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Find our Cookie</a></span></li><li><span><a href="#Sneakily-Acquire-Cookie" data-toc-modified-id="Sneakily-Acquire-Cookie-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Sneakily Acquire Cookie</a></span></li></ul></li><li><span><a href="#Sending-Data-with-POST" data-toc-modified-id="Sending-Data-with-POST-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Sending Data with POST</a></span><ul class="toc-item"><li><span><a href="#Simulating-Browser-Header" data-toc-modified-id="Simulating-Browser-Header-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Simulating Browser Header</a></span></li></ul></li></ul></div>

# Interacting with HTML Forms

Import request and BeautifulSoup libraries

In [1]:
import requests as r
from bs4 import BeautifulSoup

- Perform get request on URL and make a soup object
- Store the content of the select tag with id of 'ddlRandomness'

In [2]:
searchURL = 'http://drd.ba.ttu.edu/isqs6339/ex/l1.2/search.php'

res = r.get(searchURL)
soup = BeautifulSoup(res.content,'lxml')
res.content
ddlresult = soup.find("select", attrs={'id' : 'ddlRandomness'})

Look for all option tags and print their value attribute

In [4]:
for o in ddlresult.find_all('option'):
    print(o['value'])

NES
SNES
Playstation
XBOX


# Advanced Python Scraping

- Import libraries
- Create search URL
- Create action URL we will post to
- Create post-post URL
    -Post-post is my new favorite word

## Setup

In [9]:
import requests as r

searchURL = 'http://drd.ba.ttu.edu/isqs6339/ex/l1.2/search.php'
actionURL = 'http://drd.ba.ttu.edu/isqs6339/ex/l1.2/search_post.php?method=GET&searchInput=asdf&ddlRandomness=NES'
action_postURL = 'http://drd.ba.ttu.edu/isqs6339/ex/l1.2/search_post.php'

Do our basics:
- GET the URL, store in request object
- Get the status code
- Get content-type from headers
- Get encoding
- Store as text

In [10]:
res = r.get(searchURL)

res.status_code
res.headers['content-type']
res.encoding
res.text
res.content

b'<html>\n\t<head>\n\t\t<title>Test Search Page</title>\n\t</head>\n\t<body>\n\t\t\t\tLet\'s Search for Stuff!\n\t\t<br />\n\t\t<br />\n\t\t<ul>\n\t\t\t<li>Put in "TV" for one set.</li>\n\t\t\t<li>Put in "VCR" for set of obsolete information.</li>\n\t\t\t<li>Anything else defaults.</li>\n\t\t</ul>\n\t\t<br />\n\t\t<br />\n\t\t<fieldset>\n\t\t\t<legend>POST Method on Form</legend>\n\t\t\t\t<form action="search_post.php" method="POST">\n\t\t\t\t\t<input type="hidden" id="method" name="method" value="POST" />\n\t\t\t\t\t<ul>\n\t\t\t\t\t\t<li>Search String:  <input type="text" id="searchInput" name="searchInput" /></li>\n\t\t\t\t\t\t<li>Which Option:  <select id="ddlRandomness" name="ddlRandomness">\n\t\t\t\t\t\t\t\t\t<option value="NES">NES</option>\n\t\t\t\t\t\t\t\t\t<option value="SNES">SNES</option>\n\t\t\t\t\t\t\t\t\t<option value="Playstation">Playstation</option>\n\t\t\t\t\t\t\t\t\t<option value="XBOX">XBox</option>\n\t\t\t\t\t\t\t\t</select>\n\t\t\t\t\t</ul>\n\t\t\t\t\t<input type=

## Submit Form as Get with Cookies
- Get request from the actionURL
- This should return the file that the form is posting to
- We've been bamboozled

In [11]:
# Notice the action url and method

res_get = r.get(actionURL)

res_get.status_code
res_get.content

b'<html>\n\t<head>\n\t\t<title>Response to Input</title>\n\t</head>\n\t<body>\n\t\t\t\t\tSTOP TRYING TO SCRAPE MY SITE YOU EVIL ROBOT!\n\t\t\t</body>\n</html>\n\n\n\n\n\n\n'

### Find our Cookie
This site is expecting a cookie
- We can examine our previous request (res) to see what cookie was passed 
    - name='search_cookie'
- Compare this to our modified request (res_get) 
    - no cookies :( 

In [12]:
res.cookies

<RequestsCookieJar[Cookie(version=0, name='search_cookie', value='testsearch', port=None, port_specified=False, domain='drd.ba.ttu.edu', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=1601686454, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False)]>

In [13]:
res_get.cookies

<RequestsCookieJar[]>

### Sneakily Acquire Cookie
To get around this we can
- Set our first request cookie equal to 'search_cookies' variable
- Pass that variable into the get request attribute 'cookies'
- Success!

In [14]:
#Get the Cookies
search_cookies = res.cookies

#set cookies for call
res_get = r.get(actionURL, cookies=search_cookies)

res_get.content

b'<html>\n\t<head>\n\t\t<title>Response to Input</title>\n\t</head>\n\t<body>\n\t\t\n\t\t<fieldset>\n\t\t\t<legend>Output of Search</legend>\n\t\t\t<ul>\n\t\t\t\t\t<li>Method:  GET</li>\n\t\t\t\t\t\t<li>I love Nintendo!</li>\n\t\t\t\t\t\t\t\t<li>Browser Header: python-requests/2.22.0</li>\n\t\t\t</ul>\n\t\t</fieldset>\n\t\t\t</body>\n</html>\n\n\n\n\n\n\n'

## Sending Data with POST
- Be sure to use request.post, not get
- Create a dictionary to pass in data: 
    - Method: POST
    - searchInput: asdf
        - What goes into the search field
    - ddlRandomness: NES
- The actual post includes
    - The action postURL
    - post_data from above
    - cookies=search_cookies from previous example

In [15]:
post_data = {'method':'POST','searchInput':'asdf','ddlRandomness':'NES'}

res_post = r.post(action_postURL, data=post_data, cookies=search_cookies )

res_post.content

b'<html>\n\t<head>\n\t\t<title>Response to Input</title>\n\t</head>\n\t<body>\n\t\t\n\t\t<fieldset>\n\t\t\t<legend>Output of Search</legend>\n\t\t\t<ul>\n\t\t\t\t\t<li>Method:  POST</li>\n\t\t\t\t\t\t<li>I love Nintendo!</li>\n\t\t\t\t\t\t\n\t\t\t\t\t<li>Browser Header: python-requests/2.22.0</li>\n\t\t\t</ul>\n\t\t</fieldset>\n\t\t\t</body>\n</html>\n\n\n\n\n\n\n'

### Simulating Browser Header

Note the 'Browser Header' in the previous output
- 'python-requests'
- Websites may not want scraping from python requests
- For this reason we need to pass a simulated browser header
- Check the header in the new output

In [17]:
#Let's pass a real browser header

# Store Firefox browser agent in dictionary with user-agent as key
headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:65.0) Gecko/20100101 Firefox/65.0'}

# Notice headers=headers, we pass in our dictionary
res_post = r.post(action_postURL, data=post_data, cookies=search_cookies, headers=headers)

res_post.content

b'<html>\n\t<head>\n\t\t<title>Response to Input</title>\n\t</head>\n\t<body>\n\t\t\n\t\t<fieldset>\n\t\t\t<legend>Output of Search</legend>\n\t\t\t<ul>\n\t\t\t\t\t<li>Method:  POST</li>\n\t\t\t\t\t\t<li>I love Nintendo!</li>\n\t\t\t\t\t\t\n\t\t\t\t\t<li>Browser Header: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:65.0) Gecko/20100101 Firefox/65.0</li>\n\t\t\t</ul>\n\t\t</fieldset>\n\t\t\t</body>\n</html>\n\n\n\n\n\n\n'

We can also do this in a GET:

In [18]:
res_get = r.get(actionURL, cookies=search_cookies, headers=headers)

res_get.content

b'<html>\n\t<head>\n\t\t<title>Response to Input</title>\n\t</head>\n\t<body>\n\t\t\n\t\t<fieldset>\n\t\t\t<legend>Output of Search</legend>\n\t\t\t<ul>\n\t\t\t\t\t<li>Method:  GET</li>\n\t\t\t\t\t\t<li>I love Nintendo!</li>\n\t\t\t\t\t\t\t\t<li>Browser Header: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:65.0) Gecko/20100101 Firefox/65.0</li>\n\t\t\t</ul>\n\t\t</fieldset>\n\t\t\t</body>\n</html>\n\n\n\n\n\n\n'